# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski

### Instructions
- Sign up for RunPod here: https://runpod.io/?ref=n8yfwyum
    - Note: That's my personal referral link. Please don't use it if we are mortal enemies.

- Click *Deploy* on either `SECURE CLOUD` or `COMMUNITY CLOUD`

- Follow the rest of the instructions in this video: https://www.youtube.com/watch?v=7m__xadX0z0#t=5m33.1s

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [55]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

Cloning into 'Dreambooth-Stable-Diffusion'...


In [56]:
#BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1

Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Updating c:\dev\ai\dreambooth-stable-diffusion\src\taming-transformers clone (to revision master)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Uninstalling taming-transformers-0.0.1:
      Successfully uninstalled taming-transformers-0.0.1
  Running setup.py develop for taming-transformers


  Running command git fetch -q --tags
  Running command git reset --hard -q 24268930bf1dce879235a7fddd0b2355b84d7ea6


Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip
  Updating c:\dev\ai\dreambooth-stable-diffusion\src\clip clone (to revision main)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0
  Running setup.py develop for clip


  Running command git fetch -q --tags
  Running command git reset --hard -q d50d76daa670286dd6cacf3bcd80b5e4823fc8e1


^C
Obtaining file:///C:/dev/ai/Dreambooth-Stable-Diffusion
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: latent-diffusion
    Found existing installation: latent-diffusion 0.0.1
    Uninstalling latent-diffusion-0.0.1:
      Successfully uninstalled latent-diffusion-0.0.1
  Running setup.py develop for latent-diffusion
  Using cached protobuf-3.20.1-cp310-cp310-win_amd64.whl (903 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.


In [ ]:
## Login to stable diffusion
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\GBBarguN\.huggingface\token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
## Download the 1.4 sd model
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="CompVis/stable-diffusion-v-1-4-original",
 filename="sd-v1-4.ckpt",
 use_auth_token=True
)

Downloading:   0%|          | 0.00/4.27G [00:00<?, ?B/s]

c:\Users\GBBarguN\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GBBarguN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
## Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
import os

if os.name == 'nt':
    print("Detected MS Environment")
    !move {downloaded_model_path} model.ckpt

else:
    print("Detected Linux Environment")
    !actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
    !mv {actual_locations_of_model_blob[-1]} model.ckpt



        1 file(s) moved.


In [ ]:
#Nathan's Additional, needs pytorch with CUDA
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ---------------------------------------- 2.2/2.2 GB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 5.4/5.4 MB 85.2 MB/s eta 0:00:00
     ------------------------------------- 573.4/573.4 kB 37.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1
    Uninstalling torchvision-0.13.1:
      Successfully uninstalled torchvision-0.13.1


# Regularization Images (Skip this section if you are uploading your own or using the provided images)

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [6]:
# Estimate Runtime: ~28m

# GENERATE 200 images
!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 200 \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt "person"

# If you don't want to train against "person", change it to whatever you want above, and on some of the cells below:

Loading model from model.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sa

Global seed set to 10
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.self_attn.v_proj.bias', 'vision_model.encoder.layers.19.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.layer_norm2.bias', 'vision_model.encoder.layers.6.self_attn.out_proj.bias', 'vision_model.encoder.layers.19.layer_norm2.bias', 'vision_model.encoder.layers.14.mlp.fc2.weight', 'vision_model.encoder.layers.19.self_attn.k_proj.weight', 'vision_model.encoder.layers.4.self_attn.v_proj.weight', 'vision_model.encoder.layers.12.self_attn.k_proj.bias', 'vision_model.encoder.layers.1.mlp.fc1.bias', 'vision_model.encoder.layers.3.layer_norm2.weight', 'vision_model.encoder.layers.20.self_attn.out_proj.bias', 'vision_model.encoder.layers.14.layer_norm2.weight', 'vision_model.encoder.layers.8.self_attn.q_proj.bias', 'vision_model.encoder.layers.3.mlp.fc1.bias', 'vision_model.encoder.layers.22.self_attn.v_proj.weight',

In [17]:
# Estimate Runtime: ~9s

import os

if os.name == 'nt':
    print("Detected MS Environment")
    !powershell Compress-Archive ./outputs/ ./all_images.zip

else:
    print("Detected Linux Environment")
    !apt-get install -y zip
    !zip -r all_images.zip outputs/

# Download this file locally so you can reuse during another training on this dataset

Detected MS Environment


Compress-Archive : The archive file C:\dev\ai\Dreambooth-Stable-Diffusion\all_images.zip already exists. Use the 
-Update parameter to update the existing archive file or use the -Force parameter to overwrite the existing archive 
file.
At line:1 char:1
+ Compress-Archive ./outputs/ ./all_images.zip
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidArgument: (C:\dev\ai\Dream...\all_images.zip:String) [Compress-Archive], IOExcept 
   ion
    + FullyQualifiedErrorId : ArchiveFileExists,Compress-Archive
 


## Download pre-generated regularization images

We've created the following image sets

* man_euler - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
* man_unsplash - pictures from various photographers
* person_ddim
* woman_ddim - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0

`person_ddim` is recommended

In [20]:
# Grab the existing regularization images
# Choose the dataset that best represents what you are trying to do and matches what you used for your token
# man_euler, man_unsplash, person_ddim, woman_ddim

dataset="person_ddim"
#dataset="man_unsplash"
#dataset="woman_ddim"
#dataset="man_euler"

!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

import os

if os.name == 'nt':
    print("Detected MS Environment")
    !mkdir .\outputs\txt2img-samples\samples\{dataset}
    !move .\Stable-Diffusion-Regularization-Images-{dataset}\{dataset}\*.* outputs\txt2img-samples\samples\{dataset}
    !mkdir .\training_samples

else:
    print("Detected Linux Environment")
    !mkdir -p outputs/txt2img-samples/samples/{dataset}
    !mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* outputs/txt2img-samples/samples/{dataset}

Detected MS Environment


Cloning into 'Stable-Diffusion-Regularization-Images-man_euler'...
Updating files:  22% (277/1250)
Updating files:  23% (288/1250)
Updating files:  24% (300/1250)
Updating files:  25% (313/1250)
Updating files:  26% (325/1250)
Updating files:  27% (338/1250)
Updating files:  28% (350/1250)
Updating files:  29% (363/1250)
Updating files:  30% (375/1250)
Updating files:  31% (388/1250)
Updating files:  32% (400/1250)
Updating files:  33% (413/1250)
Updating files:  34% (425/1250)
Updating files:  35% (438/1250)
Updating files:  36% (450/1250)
Updating files:  37% (463/1250)
Updating files:  38% (475/1250)
Updating files:  39% (488/1250)
Updating files:  40% (500/1250)
Updating files:  41% (513/1250)
Updating files:  42% (525/1250)
Updating files:  42% (531/1250)
Updating files:  43% (538/1250)
Updating files:  44% (550/1250)
Updating files:  45% (563/1250)
Updating files:  46% (575/1250)
Updating files:  47% (588/1250)
Updating files:  48% (600/1250)
Updating files:  49% (613/1250)
Updat

c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00000.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00001.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00002.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00003.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00004.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00005.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00006.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regularization-Images-man_euler\man_euler\man_euler_00007.png
c:\dev\ai\Dreambooth-Stable-Diffusion\Stable-Diffusion-Regulariz

A subdirectory or file .\training_samples already exists.


# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_samples
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body 
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make (most of the time, that means no selfies).
- 

In [ ]:
#@markdown Add here the URLs to the images of the subject you are adding
urls = [
 "https://i.imgur.com/test1.png",
 "https://i.imgur.com/test2.png",
 "https://i.imgur.com/test3.png",
 "https://i.imgur.com/test4.png",
 "https://i.imgur.com/test5.png",
 # You can add additional images here -- about 20-30 images in different 
]

In [ ]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_samples"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

## Edit the personalized.py file
Execute this cell `%load ldm/data/personalized.py`

Change `joepenna` to whatever you want it to be (but keep the {})

```
training_templates_smallest = [
    'joepenna {}',
]
```

I recommend using the name of a celebrity that:
1) kinda looks like you.
2) Stable Diffusion generates well (you can check by typing their name on DreamStudio)

Then paste this at the very top of the cell:
```
%%writefile ldm/data/personalized.py
```

Then run the cell again.  This will save your changes.


In [21]:
# %load ldm/data/personalized.py
import os
import numpy as np
import PIL
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

import random

training_templates_smallest = [
    'nbarguss {}',
]

reg_templates_smallest = [
    '{}',
]

imagenet_templates_small = [

    '{}',
]

imagenet_dual_templates_small = [
    '{} with {}'
]

per_img_token_list = [
    'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'כ', 'ל', 'מ', 'נ', 'ס', 'ע', 'פ', 'צ', 'ק', 'ר', 'ש', 'ת',
]


class PersonalizedBase(Dataset):
    def __init__(self,
                 data_root,
                 size=None,
                 repeats=100,
                 interpolation="bicubic",
                 flip_p=0.0,
                 set="train",
                 placeholder_token="dog",
                 per_image_tokens=False,
                 center_crop=False,
                 mixing_prob=0.25,
                 coarse_class_text=None,
                 reg=False
                 ):

        self.data_root = data_root

        self.image_paths = [os.path.join(
            self.data_root, file_path) for file_path in os.listdir(self.data_root)]

        # self._length = len(self.image_paths)
        self.num_images = len(self.image_paths)
        self._length = self.num_images

        self.placeholder_token = placeholder_token

        self.per_image_tokens = per_image_tokens
        self.center_crop = center_crop
        self.mixing_prob = mixing_prob

        self.coarse_class_text = coarse_class_text

        if per_image_tokens:
            assert self.num_images < len(
                per_img_token_list), f"Can't use per-image tokens when the training set contains more than {len(per_img_token_list)} tokens. To enable larger sets, add more tokens to 'per_img_token_list'."

        if set == "train":
            self._length = self.num_images * repeats

        self.size = size
        self.interpolation = {"linear": PIL.Image.LINEAR,
                              "bilinear": PIL.Image.BILINEAR,
                              "bicubic": PIL.Image.BICUBIC,
                              "lanczos": PIL.Image.LANCZOS,
                              }[interpolation]
        self.flip = transforms.RandomHorizontalFlip(p=flip_p)
        self.reg = reg

    def __len__(self):
        return self._length

    def __getitem__(self, i):
        example = {}
        image = Image.open(self.image_paths[i % self.num_images])

        if not image.mode == "RGB":
            image = image.convert("RGB")

        placeholder_string = self.placeholder_token
        if self.coarse_class_text:
            placeholder_string = f"{self.coarse_class_text} {placeholder_string}"

        if not self.reg:
            text = random.choice(training_templates_smallest).format(
                placeholder_string)
        else:
            text = random.choice(reg_templates_smallest).format(
                placeholder_string)

        example["caption"] = text

        # default to score-sde preprocessing
        img = np.array(image).astype(np.uint8)

        if self.center_crop:
            crop = min(img.shape[0], img.shape[1])
            h, w, = img.shape[0], img.shape[1]
            img = img[(h - crop) // 2:(h + crop) // 2,
                      (w - crop) // 2:(w + crop) // 2]

        image = Image.fromarray(img)
        if self.size is not None:
            image = image.resize((self.size, self.size),
                                 resample=self.interpolation)

        image = self.flip(image)
        image = np.array(image).astype(np.uint8)
        example["image"] = (image / 127.5 - 1.0).astype(np.float32)
        return example


In [58]:
# START THE TRAINING
project_name = "NathanV2"

# MAX STEPS
# It's how long you want your training to go.
# If you're seeing this message, I'm literally at my computer right now fixing this up:
max_training_steps = 1000
class_word = "person"  # << match this word to the class word from regularization images above

dataset="person_ddim"
#dataset="man_unsplash"
#dataset="woman_ddim"
#dataset="man_euler"


import os

if os.name == 'nt':
    print("Detected MS Environment")

    reg_data_root = os.path.join(os.getcwd(),"outputs\\txt2img-samples\\samples\\",dataset,) + "\\"
    data_root = os.path.join(os.getcwd(),"training_samples")
    !del /F /Q .\training_samples\.ipynb_checkpoints

else:
    print("Detected Linux Environment")

    reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples/" + dataset
    data_root = "/workspace/Dreambooth-Stable-Diffusion/training_samples"
    !rm -rf training_samples/.ipynb_checkpoints

!echo Using reg_data_root of: "{reg_data_root}"
!echo Training on user data here: "{data_root}"

!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root {reg_data_root} \
 -n {project_name} \
 --gpus 0, \
 --data_root "{data_root}" \
 --max_training_steps {max_training_steps} \
 --class_word {class_word} \
 --no-test

Detected MS Environment


Could Not Find c:\dev\ai\Dreambooth-Stable-Diffusion\training_samples\.ipynb_checkpoints


Using reg_data_root of: "c:\dev\ai\Dreambooth-Stable-Diffusion\outputs\txt2img-samples\samples\person_ddim\"
Training on user data here: "c:\dev\ai\Dreambooth-Stable-Diffusion\training_samples"


## Pruning (12GB to 2GB)
We are working on having this happen automatically (TODO: PR's welcome)

In [ ]:
directory_paths = !ls -d logs/*

In [ ]:
# This version should automatically prune around 10GB from the ckpt file
#last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"

!python "prune_ckpt.py" --ckpt "C:\\dev\\ai\\Dreambooth-Stable-Diffusion\\logs\\training_samples2022-10-04T12-07-16_project_name\\checkpoints\\last.ckpt"

prunin' in path: C:\\dev\\ai\\Dreambooth-Stable-Diffusion\\logs\\training_samples2022-10-04T12-07-16_project_name\\checkpoints\\last.ckpt
dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])
removing optimizer states for path C:\\dev\\ai\\Dreambooth-Stable-Diffusion\\logs\\training_samples2022-10-04T12-07-16_project_name\\checkpoints\\last.ckpt
This is global step 167.
saving pruned checkpoint at: C:\\dev\\ai\\Dreambooth-Stable-Diffusion\\logs\\training_samples2022-10-04T12-07-16_project_name\\checkpoints\\last-pruned.ckpt
New ckpt size: 2.13 GB. Saved 9.99 GB by removing optimizer states


In [ ]:
last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
training_samples = !ls training_samples
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_samples)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + class_word + "_class_word.ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{file_name}

In [ ]:
# Download your trained model file from `trained_models` and use in your favorite Stable Diffusion repo!

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [5]:
#!python scripts/stable_txt2img.py \
# --ddim_eta 0.0 \
# --n_samples 1 \
# --n_iter 4 \
# --scale 7.0 \
# --ddim_steps 50 \
# --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/" + {file_name} \
# --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"


 

!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 8 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "C:\\dev\\ai\\Dreambooth-Stable-Diffusion\\trained_models\\johnrequa-last-pruned.ckpt"\
 --prompt "johnrequa person portrait fantasy wizard painting"

Loading model from C:\\dev\\ai\\Dreambooth-Stable-Diffusion\\trained_models\\johnrequa-last-pruned.ckpt
Global Step: 167
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64),

Global seed set to 42
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.3.self_attn.q_proj.bias', 'vision_model.encoder.layers.1.layer_norm2.weight', 'vision_model.encoder.layers.15.self_attn.q_proj.weight', 'vision_model.encoder.layers.5.mlp.fc1.weight', 'vision_model.encoder.layers.11.self_attn.q_proj.weight', 'vision_model.encoder.layers.13.self_attn.v_proj.weight', 'vision_model.encoder.layers.10.self_attn.q_proj.weight', 'vision_model.encoder.layers.11.self_attn.k_proj.weight', 'vision_model.encoder.layers.7.layer_norm2.weight', 'vision_model.encoder.layers.7.self_attn.out_proj.weight', 'vision_model.encoder.layers.18.mlp.fc2.weight', 'vision_model.encoder.layers.20.self_attn.v_proj.weight', 'vision_model.encoder.layers.13.self_attn.out_proj.weight', 'vision_model.encoder.layers.15.self_attn.v_proj.bias', 'vision_model.encoder.layers.17.self_attn.v_proj.weight', 'vision_model.encoder.l